In [ ]:
!pip install trieste

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.6/253.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.8/379.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.6/111.6 kB 13.3 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
 

In [ ]:
from trieste.space import Box
import tensorflow as tf
import numpy as np

search_space = Box([-100]*20, [100]*20)  # define the search space directly

In [ ]:
import trieste

In [ ]:
n = 20 #Cantidad de coeficientes
N = 15*n #Numero de frecuencias
#Vector columna de frecuencias
w = tf.reshape(tf.linspace(0.0,3.14516,N),(N,1))
# Respuesta en frecuencia del filtro deseado
D = 5.25
Hdes = tf.exp(tf.complex(0.0, -1.0)*tf.complex(D*w,0.0))
# Construccion de la matriz A
A = tf.exp( tf.complex(0.0, -1.0) * tf.cast(tf.experimental.numpy.kron(w, tf.experimental.numpy.arange(n)), dtype = 'complex64') )
#Extraer parte real e imaginaria
Hdes_R = tf.math.real(Hdes)
Hdes_I = tf.math.imag(Hdes)
A_R = tf.math.real(A)
A_I = tf.math.imag(A)

In [ ]:
def ObjFun(h):
  if h.dtype == tf.float64:
    h = tf.cast(h,tf.float32)
  if len(h.shape)==1:
    ht = tf.reshape(h,[1,h.shape[0],1])
  elif len(h.shape)==2:
    ht = tf.reshape(h,[h.shape[0],h.shape[1],1])
  else:
    ht = h.copy()
  A_Rt = tf.repeat(tf.reshape(A_R,[1,A_R.shape[0],A_R.shape[1]]),ht.shape[0],axis=0)
  A_It = tf.repeat(tf.reshape(A_I,[1,A_I.shape[0],A_I.shape[1]]),ht.shape[0],axis=0)
  Hdes_Rt = tf.repeat(tf.reshape(Hdes_R,[1,Hdes_R.shape[0],Hdes_R.shape[1]]),ht.shape[0],axis=0)
  Hdes_It = tf.repeat(tf.reshape(Hdes_I,[1,Hdes_I.shape[0],Hdes_I.shape[1]]),ht.shape[0],axis=0)
  res = tf.math.reduce_sum( tf.square(tf.linalg.matmul(A_Rt,ht) - Hdes_Rt) +\
                            tf.square(tf.linalg.matmul(A_It,ht)- Hdes_It), axis = 1)
  return tf.cast(res, tf.float64)

In [ ]:
h = tf.random.uniform(shape=[5,20])
ObjFun(h)

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[1644.82019043],
       [1902.90490723],
       [1687.85009766],
       [1912.24353027],
       [2421.31933594]])>

In [ ]:
observer = trieste.objectives.utils.mk_observer(ObjFun)

num_initial_points = 5
initial_query_points = search_space.sample_sobol(num_initial_points)
initial_data = observer(initial_query_points)

In [ ]:
initial_data

Dataset(query_points=<tf.Tensor: shape=(5, 20), dtype=float64, numpy=
array([[  2.46276855,  84.5489502 , -83.3770752 , -76.62658691,
        -18.36853027, -19.40612793,  69.15588379,  11.61804199,
        -26.18713379,  99.80163574, -68.40515137, -18.53942871,
         10.65979004,  51.47399902, -83.12683105, -60.1348877 ,
         78.14025879, -62.68005371, -97.97668457,  19.52209473],
       [ 52.46276855, -65.4510498 ,  66.6229248 ,  73.37341309,
        -68.36853027, -69.40612793, -80.84411621,  61.61804199,
        -76.18713379,  49.80163574, -18.40515137, -68.53942871,
        -39.34020996, -98.52600098, -33.12683105,  89.8651123 ,
         28.14025879,  87.31994629, -47.97668457, -30.47790527],
       [-47.53723145,  34.5489502 , -33.3770752 , -26.62658691,
         31.63146973,  30.59387207,  19.15588379, -38.38195801,
         23.81286621, -50.19836426,  81.59484863,  31.46057129,
         60.65979004,   1.47399902,  66.87316895, -10.1348877 ,
        -71.85974121, -12.680053

In [ ]:
import gpflow
import tensorflow_probability as tfp

from trieste.models.gpflow import GaussianProcessRegression


def build_model(data):
    variance = tf.math.reduce_variance(data.observations)
    kernel = gpflow.kernels.Matern52(variance=variance, lengthscales=[0.2, 0.2])
    prior_scale = tf.cast(1.0, dtype=tf.float64)
    kernel.variance.prior = tfp.distributions.LogNormal(
        tf.cast(-2.0, dtype=tf.float64), prior_scale
    )
    kernel.lengthscales.prior = tfp.distributions.LogNormal(
        tf.math.log(kernel.lengthscales), prior_scale
    )
    gpr = gpflow.models.GPR(data.astuple(), kernel, noise_variance=1e-5)
    gpflow.set_trainable(gpr.likelihood, False)

    return GaussianProcessRegression(gpr, num_kernel_samples=100)


model = build_model(initial_data)

In [ ]:
from trieste.models.gpflow import build_gpr

gpflow_model = build_gpr(initial_data, search_space, likelihood_variance=1e-7)
model = GaussianProcessRegression(gpflow_model, num_kernel_samples=100)

AttributeError: ignored